In [1]:
import requests

# API key
API_KEY = "6XdJHAodRZCtiXxuFXovlE1w73QhyH7cMOf0xRmX"

# Competitions endpoint
url = "https://api.sportradar.com/tennis/trial/v3/en/competitions.json"

# Call API
response = requests.get(url, params={"api_key": API_KEY})

# Check status
print("Status Code:", response.status_code)

# Print a sample response
if response.status_code == 200:
    data = response.json()
    print("Sample Competition:", data['competitions'][0])
else:
    print("Error:", response.text)

Status Code: 200
Sample Competition: {'id': 'sr:competition:620', 'name': 'Hopman Cup', 'type': 'mixed', 'gender': 'mixed', 'category': {'id': 'sr:category:181', 'name': 'Hopman Cup'}}


In [2]:
import mysql.connector

# Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",          # change if your username is different
    password="Veerendra@06",          # add your MySQL password if any
    database="tennis_data"
)

cursor = conn.cursor()
print("Connected to MySQL!")


Connected to MySQL!


In [3]:
# Extract categories from competitions data
categories_data = {}

for comp in data['competitions']:
    cat = comp['category']
    cat_id = cat['id']
    cat_name = cat['name']
    
    if cat_id not in categories_data:
        categories_data[cat_id] = cat_name

# Insert into 'categories' table
for cat_id, cat_name in categories_data.items():
    query = "INSERT INTO categories (category_id, category_name) VALUES (%s, %s)"
    values = (cat_id, cat_name)
    cursor.execute(query, values)

conn.commit()
print(f"Inserted {len(categories_data)} categories successfully!")


Inserted 18 categories successfully!


In [4]:
# Insert into competitions table
inserted = 0

for comp in data['competitions']:
    comp_id = comp.get('id')
    comp_name = comp.get('name')
    parent_id = comp.get('parent_id')  # can be None
    comp_type = comp.get('type')
    gender = comp.get('gender')
    cat_id = comp['category']['id']  # foreign key from categories

    query = """
        INSERT INTO competitions (
            competition_id, competition_name, parent_id, type, gender, category_id
        ) VALUES (%s, %s, %s, %s, %s, %s)
    """
    values = (comp_id, comp_name, parent_id, comp_type, gender, cat_id)
    
    cursor.execute(query, values)
    inserted += 1

conn.commit()
print(f"Inserted {inserted} competitions successfully!")


Inserted 6161 competitions successfully!


In [5]:
# Complexes endpoint
complex_url = "https://api.sportradar.com/tennis/trial/v3/en/complexes.json"

# Call the API
response = requests.get(complex_url, params={"api_key": API_KEY})

# Check response
print("Status Code:", response.status_code)

if response.status_code == 200:
    complex_data = response.json()
    print("Sample Complex:", complex_data['complexes'][0])
else:
    print("Error:", response.text)


Status Code: 200
Sample Complex: {'id': 'sr:complex:705', 'name': 'Nacional', 'venues': [{'id': 'sr:venue:70045', 'name': 'Cancha Central', 'city_name': 'Santiago', 'country_name': 'Chile', 'country_code': 'CHL', 'timezone': 'America/Santiago'}]}


In [6]:
# Insert into 'complexes' table
complex_list = complex_data['complexes']
inserted_complexes = 0

for item in complex_list:
    complex_id = item.get('id')
    complex_name = item.get('name')

    query = "INSERT INTO complexes (complex_id, complex_name) VALUES (%s, %s)"
    values = (complex_id, complex_name)
    
    cursor.execute(query, values)
    inserted_complexes += 1

conn.commit()
print(f"Inserted {inserted_complexes} complexes successfully!")


Inserted 680 complexes successfully!


In [7]:
# Insert venues linked to each complex
inserted_venues = 0

for complex_item in complex_list:
    complex_id = complex_item.get('id')
    venues = complex_item.get('venues', [])
    
    for v in venues:
        venue_id = v.get('id')
        venue_name = v.get('name')
        city_name = v.get('city_name')
        country_name = v.get('country_name')
        country_code = v.get('country_code')
        timezone = v.get('timezone')

        query = """
            INSERT INTO venues (
                venue_id, venue_name, city_name, country_name,
                country_code, timezone, complex_id
            ) VALUES (%s, %s, %s, %s, %s, %s, %s)
        """
        values = (
            venue_id, venue_name, city_name, country_name,
            country_code, timezone, complex_id
        )

        cursor.execute(query, values)
        inserted_venues += 1

conn.commit()
print(f"Inserted {inserted_venues} venues successfully!")


Inserted 3522 venues successfully!


In [11]:
# Sample data simulating the API response
ranking_data = {
    "rankings": [
        {
            "rank": 1,
            "movement": 0,
            "points": 8000,
            "competitions_played": 18,
            "competitor": {
                "id": "sr:competitor:1001",
                "name": "John Doe",
                "country": "USA",
                "country_code": "USA",
                "abbreviation": "JDO"
            }
        },
        {
            "rank": 2,
            "movement": 1,
            "points": 7800,
            "competitions_played": 19,
            "competitor": {
                "id": "sr:competitor:1002",
                "name": "Carlos Smith",
                "country": "Spain",
                "country_code": "ESP",
                "abbreviation": "CSM"
            }
        },
        {
            "rank": 3,
            "movement": -1,
            "points": 7500,
            "competitions_played": 20,
            "competitor": {
                "id": "sr:competitor:1003",
                "name": "Liam Zhang",
                "country": "China",
                "country_code": "CHN",
                "abbreviation": "LZH"
            }
        }
    ]
}


In [12]:
# Insert competitors into 'competitors' table
inserted_competitors = 0

for player in ranking_data["rankings"]:
    comp = player["competitor"]
    comp_id = comp["id"]
    name = comp["name"]
    country = comp["country"]
    country_code = comp["country_code"]
    abbreviation = comp["abbreviation"]

    query = """
        INSERT INTO competitors (competitor_id, name, country, country_code, abbreviation)
        VALUES (%s, %s, %s, %s, %s)
    """
    values = (comp_id, name, country, country_code, abbreviation)

    cursor.execute(query, values)
    inserted_competitors += 1

conn.commit()
print(f"Inserted {inserted_competitors} competitors successfully!")


Inserted 3 competitors successfully!


In [14]:
# Insert rankings into 'competitor_rankings' table
inserted_rankings = 0

for player in ranking_data["rankings"]:
    rank = player["rank"]
    movement = player["movement"]
    points = player["points"]
    competitions_played = player["competitions_played"]
    competitor_id = player["competitor"]["id"]

    query = """
        INSERT INTO competitor_rankings (
            `rank`, movement, points, competitions_played, competitor_id
        ) VALUES (%s, %s, %s, %s, %s)
    """
    values = (rank, movement, points, competitions_played, competitor_id)

    cursor.execute(query, values)
    inserted_rankings += 1

conn.commit()
print(f"Inserted {inserted_rankings} competitor rankings successfully!")


Inserted 3 competitor rankings successfully!
